In [2]:
import pandas as pd
from pathlib import Path
import os

In [3]:
DATA_DIR = Path("data", "ijcnlp_dailydialog", "train")
data = pd.read_csv(Path(DATA_DIR, "dialogues_train.txt"),  delimiter = "\n", names = ["dialogues"])

In [4]:
def seputterances(row):
    try:
        row = row.split("__eou__")
        row = row[:-1]
        return row
    except:
        return row

data["dialogues"] = data["dialogues"].apply(seputterances)

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

/raid/cs18resch11003/anaconda3/envs/dialogpt/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<|endoftext|>


In [6]:
num_context = 3

utterance = []
history = []

for i in data.index:

    row = data["dialogues"][i]

    for idx  in range(len(row)):

        if idx != 0:
            
            utterance.append(row[idx])

            counter = 1
            _history = ""
            
            for k in range(idx-1, -1, -1):
                if counter <= num_context:
                    _history = _history + row[k]
                    counter +=1
                else:
                    break
                _history = _history + tokenizer.eos_token
            history.append(_history)
        else:
            continue

In [7]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
max_len = 32

In [91]:
import torch

input_ids = []

attention_masks = []

labels = []

for i in range(len(utterance)):
    
    encoded_utterance = tokenizer.encode_plus(utterance[i].lower() + tokenizer.eos_token, max_length = max_len, padding= "max_length", truncation = True, return_tensors = "pt")
    
    encoded_history = tokenizer.encode_plus(history[i].lower(), max_length = max_len, truncation = True, padding= "max_length", return_tensors = "pt")

    ids = torch.cat([encoded_utterance["input_ids"][0], encoded_history["input_ids"][0]], dim=0).reshape(1,max_len*2)
    mask = torch.cat([encoded_utterance["attention_mask"][0], encoded_history["attention_mask"][0]], dim=0).reshape(1,max_len*2)
    label = torch.cat([torch.full((max_len,), -100), torch.full((max_len,), 1)], dim = 0).reshape(1, max_len*2)

    input_ids.append(ids)

    attention_masks.append(mask)
    labels.append(label)
    if i == 10:
        break

In [92]:
input_ids = torch.cat(input_ids, dim = 0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.cat(labels, dim=0)


In [93]:
from torch.utils.data import TensorDataset

dataset = TensorDataset(input_ids, attention_masks, labels)

In [94]:
from torch.utils.data import DataLoader, RandomSampler

dataloader = DataLoader(
            dataset,
            sampler = RandomSampler(dataset),
            batch_size = 8
        )

In [97]:
for i, batch in enumerate(dataloader):
    print(batch[0], batch[1], batch[2])


tensor([[  345,   760,   326,   318, 29850,   475,   318,  1107,   407,   922,
           329,   674, 13547,   764,   220, 50256, 50257, 50257, 50257, 50257,
         50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
         50257, 50257, 16706,   837,   474,   320,   837,   703,   546,  1016,
           329,   257,  1178, 16800,   706,  8073,  5633,   220, 50256, 50257,
         50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
         50257, 50257, 50257, 50257],
        [ 5238,  1049,   284,   502,  5145,   611,   484,   389,  4684,   837,
           356,   714,  1265,   606,   284,   467, 15360,   351,   514,    13,
          5562,   318,  6275,  5517,   290,  1257,   837,  1165,   764,   220,
         50256, 50257,   326,   338,   257,   922,  2126,   764,  1312,  3285,
           285,   560,   290,   264,   453,  1690,   467,   612,   284,   711,
         29400,    79,   506,    13, 28998,   356,   460,   787,   257,  1440,
         11246

In [117]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

In [118]:
dataset = {"S1":[], "S2": []}


for i in range(len(data)):
    for j in range(len(data["dialogues"][i])):
        dataset["S1"].append(data["dialogues"][i][j])
        try:
            dataset["S2"].append(data["dialogues"][i][j+1])
        except:
            dataset["S2"].append(tokenizer.eos_token)

In [119]:
tokenizer.pad_token = tokenizer.eos_token

tokenized_dataset = tokenizer(
    dataset["S1"],
    dataset["S2"],
    padding=True,
    truncation=True,
    return_tensors = "pt"
)

tokenized_dataset

{'input_ids': tensor([[25515,   837,  5395,  ..., 50256, 50256, 50256],
        [  921,   760,   326,  ..., 50256, 50256, 50256],
        [ 1867,   466,   345,  ..., 50256, 50256, 50256],
        ...,
        [  314,   761,  2546,  ..., 50256, 50256, 50256],
        [23722,   314,   423,  ..., 50256, 50256, 50256],
        [16805,   764,   314,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [3]:
import pickle 

dataset = load_dataset('daily_dialog')

Using custom data configuration default
Reusing dataset daily_dialog (/home/cs18resch11003/.cache/huggingface/datasets/daily_dialog/default/1.0.0/1d0a58c7f2a4dab5ed9d01dbde8e55e0058e589ab81fce5c2df929ea810eabcd)
100%|██████████| 3/3 [00:00<00:00, 469.06it/s]


In [7]:
train_dataset = dataset["train"]
train_dataset

Dataset({
    features: ['dialog', 'act', 'emotion'],
    num_rows: 11118
})

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

In [11]:
tokenizer.pad_token = tokenizer.eos_token

tokenized_dataset = tokenizer(
    train_dataset["dialog"][0],
    train_dataset["dialog"][0],
    padding=True,
    truncation=True,
    return_tensors = "pt"
)

In [39]:
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    print(example["dialog"][0])
    exit

In [40]:
tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_datasets

 50%|█████     | 6/12 [00:00<00:00, 28.05ba/s]

['Say , Jim , how about going for a few beers after dinner ? ', ' You know that is tempting but is really not good for our fitness . ', ' What do you mean ? It will help us to relax . ', " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ", " I guess you are right.But what shall we do ? I don't feel like sitting at home . ", ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ', " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ", ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ', " Good.Let ' s go now . ", ' All right . ']
['Great party , isn ’ t it ? ', ' Yeah , really . ', ' By the way , my name is Liu Wei . ', ' Hi , I ’ m Susan Marshall , You can call me Susan.Nice to meet you . ', ' What do you do , Susan ? ', ' Well , I ’ m a college student.Ho

100%|██████████| 12/12 [00:00<00:00, 30.39ba/s]

['Do you have this design with only one breast pocket ? ', ' Let me see.Oh , we have the design but not the same color as this one . ', " It's a pity . I'm afraid that's too loud for me.Thank you . ", " You're welcome . "]
['I have been getting headaches almost every day lately . ', ' Have you just started getting a lot of headaches ? ', ' I never had very many headaches before , but the last few weeks I have been getting a lot of them . ', ' Have you had any unusually stressful situations in your life lately ? ', ' My mother just passed away last Tuesday . ', ' I ’ m sorry . How about sleep ? Are you getting enough rest ? ', ' I have been working really hard , and sleep has not been a priority . ', ' Have you bumped your head or fallen lately ? ', ' No , I haven ’ t hit my head . ', ' I am going to send you to a neurologist for a few tests . ']
['May I help you ? Miss ! ', " Yes . I'd like to look at lipstick and eye shadow . ", ' What color set do you prefer ? ', ' Well , brown . ', 

Dataset({
    features: ['dialog', 'act', 'emotion'],
    num_rows: 11118
})

In [123]:
import os.path
os.path.isfile(Path("data", "inturn_conversations.pkl")) 

True

In [105]:
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

In [108]:
import torch
from transformers import AdamW
from transformers import get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3

num_batch = 2

num_training_steps = int(num_epochs * len(tokenized_dataset["input_ids"])/4)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

print(num_training_steps, device)

77235 cuda


In [109]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()

for epoch in range(num_epochs):
    for i in range(len(tokenized_dataset["input_ids"])):
        inputs= {}

        ids = tokenized_dataset["input_ids"][:num_batch]
        mask = tokenized_dataset["attention_mask"][:num_batch]

        inputs["input_ids"] = ids.to(device)
        inputs["attention_mask"] = mask.to(device)

        outputs = model(**inputs, labels = inputs["input_ids"])
        print(outputs)

        loss = outputs.loss
        
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        
        progress_bar.update(1)
        
        break


  0%|          | 0/77235 [00:10<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 15.90 GiB total capacity; 4.54 GiB already allocated; 23.75 MiB free; 4.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF